<a href="https://colab.research.google.com/github/AbdullahAloyaydi/MySubmit/blob/main/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxlsb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd     # provides high-performance, easy to use structures and data analysis tools
import pyxlsb           # Excel extention to read xlsb files (the input file)
import numpy as np      # provides fast mathematical computation on arrays and matrices

In [9]:
dataframe = pd.read_excel("stc TV Data Set_T1.xlsb",sheet_name="Final_Dataset")
# Please make a copy of dataset if you are going to work directly and make changes on the dataset
# you can use   
df=dataframe.copy()

In [10]:
# check the data shape
dataframe.shape

(1048575, 13)

In [11]:
dataframe.head()

,Column1,date_,user_id_maped,program_name,duration_seconds,program_class,season,episode,program_desc,program_genre,series_title,hd,original_name
0,1,42882,26138,100 treets,40,MOVIE,0,0,Drama Movie100 Streets,Drama,0,0,100 treets
1,3,42876,7946,Moana,17,MOVIE,0,0,Animation MovieMoana (HD),Animation,0,1,Moana
2,4,42957,7418,The Mermaid Princess,8,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess
3,5,42942,19307,The Mermaid Princess,76,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess
4,7,42923,15860,Churchill,87,MOVIE,0,0,Biography MovieChurchill (HD),Biography,0,1,Churchill


In [12]:
# check if any column has null value in the dataset
dataframe.isnull().any()

Column1             False
date_               False
user_id_maped       False
program_name        False
duration_seconds    False
program_class       False
season              False
episode             False
program_desc         True
program_genre       False
series_title        False
hd                  False
original_name       False
dtype: bool

In [27]:
# Here we try to get the most watched movies (Total Views / Total Users Views / Total watch time)
# For series we concatenated the Session episode to differentiate between episodes 
grouped=df.copy()
grouped.loc[grouped['program_class'] == 'SERIES/EPISODES', 'program_name'] = grouped['program_name']+'_SE'+grouped['season'].astype(str)+'_EP'+grouped['episode'].astype(str)
grouped = grouped.groupby(['program_name','program_class'])\
.agg({'user_id_maped': [('co1', 'nunique'),('co2', 'count')],\
      'duration_seconds': [('co3', 'sum')] }).reset_index()
grouped.columns = ['program_name','program_class','No of Users who Watched', 'No of watches', 'Total watch time in seconds']
grouped['Total watch time in houres']=grouped['Total watch time in seconds']/3600
grouped = grouped.drop(columns=['Total watch time in seconds'])
grouped = grouped.sort_values(by=['Total watch time in houres', 'No of watches','No of Users who Watched'], ascending=False).reset_index(drop=True)

In [15]:
# show the result
grouped.head(40)

,program_name,program_class,No of Users who Watched,No of watches,Total watch time in houres
0,The Boss Baby,MOVIE,3348,23625,2889.027778
1,The Expendables,MOVIE,853,2119,1961.159444
2,Moana,MOVIE,2112,7521,1540.544444
3,Trolls,MOVIE,2584,13609,1512.580278
4,Bean,MOVIE,949,3617,1423.955000
5,The murfs,MOVIE,726,2438,1190.363889
6,The Amazing pider-Man,MOVIE,650,1679,1157.005833
7,Cloudy With a Chance of Meatballs,MOVIE,683,2076,948.674722
8,Hotel Transylvania,MOVIE,375,1387,905.437778
9,The Amazing pider-Man,MOVIE,443,1087,782.467500


In [16]:
# we import Visualization libraries 
# you can ignore and use any other graphing libraries 
import matplotlib.pyplot as plt # a comprehensive library for creating static, animated, and interactive visualizations
import plotly #a graphing library makes interactive, publication-quality graphs. Examples of how to make line plots, scatter plots, area charts, bar charts, error bars, box plots, histograms, heatmaps, subplots, multiple-axes, polar charts, and bubble charts.
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [17]:
# plot top 10 Programs 
fig = px.pie(grouped.head(10), values='Total watch time in houres', names='program_name',\
             hover_data=['program_class'],title='top 10 programs in total watch time in houres')
fig.show()

In [22]:
grouped=df.copy()
grouped = grouped.groupby('program_class')\
.agg({'user_id_maped': [('co1', 'nunique'),('co2', 'count')],\
      'duration_seconds': [('co3', 'sum')] }).reset_index()
grouped.columns = ['program_class','No of Users who Watched', 'No of watches', 'Total watch time in seconds']
grouped['Total watch time in houres']=grouped['Total watch time in seconds']/3600
grouped = grouped.drop(columns=['Total watch time in seconds'])
grouped = grouped.sort_values(by=['Total watch time in houres', 'No of watches','No of Users who Watched'], ascending=False).reset_index(drop=True)


In [19]:
# show the result
grouped.head()

,program_class,No of Users who Watched,No of watches,Total watch time in houres
0,SERIES/EPISODES,3901,560174,255097.787500
1,MOVIE,11355,488401,103444.145556


In [20]:
# plot the total watch time against total number of users and report your findings
fig = px.pie(grouped, values='Total watch time in houres', names='program_class',\
             hover_data=['program_class'],title='Total duration spent by program_class')
fig2 = px.pie(grouped, values='No of Users who Watched', names='program_class',\
             hover_data=['program_class'],title='Total Users watching by program_class')

fig.update_traces(sort=False)
fig2.update_traces(sort=False)
fig.show()
fig2.show()

In [38]:
#study the relation and user's behaviour Against HD flag, feel free to include any comparison 
grouped=df.copy()
grouped.loc[grouped['program_class'] == 'SERIES/EPISODES', 'program_name'] = grouped['program_name']+'_SE'+grouped['season'].astype(str)+'_EP'+grouped['episode'].astype(str)
grouped = grouped.groupby(['program_name','program_class','program_genre','hd'])\
.agg({'user_id_maped': [('co1', 'nunique'),('co2', 'count')] }).reset_index()
grouped.columns = ['program_name','program_class','program_genre','hd','No of Users who Watched', 'No of watches']
grouped = grouped.sort_values(by=['No of watches','No of Users who Watched'], ascending=False).reset_index(drop=True)

In [39]:
# show the result
grouped.head(40)

,program_name,program_class,program_genre,hd,No of Users who Watched,No of watches
0,The Boss Baby,MOVIE,Animation,1,3348,23625
1,Trolls,MOVIE,Animation,1,2584,13609
2,The Adventures of Petey and Friends,MOVIE,Animation,1,1679,9510
3,Moana,MOVIE,Animation,1,2112,7521
4,Baywatch,MOVIE,Comedy,1,2030,7332
5,The Mermaid Princess,MOVIE,Animation,1,2065,7109
6,Ferdinand,MOVIE,Animation,1,1276,6811
7,An Inconvenient equel: Truth to Power,MOVIE,Documentary,1,933,6285
8,Collateral Beauty,MOVIE,Drama,1,1702,6147
9,Inside,MOVIE,Horror,1,1586,6050
